In [1]:
from datetime import datetime
import pandas as pd
import pandas_datareader.data as web

def getStock(Stock):
    stock_df = web.DataReader(Stock, 'yahoo', start_time, now_time)
    return stock_df
if __name__ == '__main__':
    now_time = datetime.now()
    start_time = datetime(now_time.year - 8, now_time.month, now_time.day)
    stock_df = getStock('TSLA')

In [2]:
stock_df.dropna()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2014-05-27,42.773998,41.540001,41.703999,42.312000,26705500.0,42.312000
2014-05-28,42.554001,41.051998,42.004002,42.048000,27475500.0,42.048000
2014-05-29,42.498001,41.543999,42.113998,42.048000,18462500.0,42.048000
2014-05-30,42.959999,41.403999,42.060001,41.554001,27905500.0,41.554001
2014-06-02,41.869999,40.334000,41.466000,40.939999,23340500.0,40.939999
...,...,...,...,...,...,...
2022-05-18,760.500000,700.809998,744.520020,709.809998,29270600.0,709.809998
2022-05-19,734.000000,694.109985,707.000000,709.419983,30098900.0,709.419983
2022-05-20,721.580017,633.000000,713.989990,663.900024,48190600.0,663.900024


In [3]:
import numpy as np
from scipy.stats import norm
import scipy.optimize as opt
import pandas as pd
import datetime
import time
from arch import arch_model
import matplotlib.pyplot as plt
from numba import jit
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
import warnings
#import plotly.express as px
#import plotly.graph_objects as go
import math
from datetime import timedelta
warnings.filterwarnings('ignore')

In [4]:
# stock_df.reset_index(inplace=True)
stock_df.sort_values(by=['Date'],ascending=False)

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2022-05-24,653.919983,620.570007,653.530029,628.159973,29498700.0,628.159973
2022-05-23,679.960022,638.059998,655.020020,674.900024,29634500.0,674.900024
2022-05-20,721.580017,633.000000,713.989990,663.900024,48190600.0,663.900024
2022-05-19,734.000000,694.109985,707.000000,709.419983,30098900.0,709.419983
2022-05-18,760.500000,700.809998,744.520020,709.809998,29270600.0,709.809998
...,...,...,...,...,...,...
2014-06-02,41.869999,40.334000,41.466000,40.939999,23340500.0,40.939999
2014-05-30,42.959999,41.403999,42.060001,41.554001,27905500.0,41.554001
2014-05-29,42.498001,41.543999,42.113998,42.048000,18462500.0,42.048000


In [5]:
ret = 100 * (stock_df.pct_change()[1:]['Adj Close'])
realized_vol = ret.rolling(5).std()

In [6]:
realized_vol

Date
2014-05-28         NaN
2014-05-29         NaN
2014-05-30         NaN
2014-06-02         NaN
2014-06-03    0.701887
                ...   
2022-05-18    5.897573
2022-05-19    5.898081
2022-05-20    5.223651
2022-05-23    5.201530
2022-05-24    4.164848
Name: Adj Close, Length: 2014, dtype: float64

In [ ]:
newnames = {'Adj Close':'Realized Volatility'} # For Visualizations
fig = px.line(realized_vol, template='plotly_white', labels={'value':'Volatility'}, title='Realized Volatility')
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
fig.update_layout(
    title_x=0.5,
    legend_title="Legend"
)
fig.show()

In [7]:

period_volatility = {}
daily_volatility = ret.std()
print('Daily volatility: ', '{:.2f}%'.format(daily_volatility))
period_volatility['Daily Volatility'] = round(daily_volatility, 2)

monthly_volatility = math.sqrt(21) * daily_volatility
print ('Monthly volatility: ', '{:.2f}%'.format(monthly_volatility))
period_volatility['Monthly Volatility'] = round(monthly_volatility, 2)

annual_volatility = math.sqrt(252) * daily_volatility
print ('Annual volatility: ', '{:.2f}%'.format(annual_volatility ))
period_volatility['Annual Volatility'] = round(annual_volatility, 2)

Daily volatility:  3.47%
Monthly volatility:  15.89%
Annual volatility:  55.05%


In [8]:
n = 300
split_date = ret.iloc[-n:].index

In [ ]:

retv = ret.values

fig = px.line(x=stock_df.index[1:], y=ret, template='plotly_white', labels={'x':'Date', 'y':'Daily Returns'}, 
              title='Volatility clustering')
fig.update_layout(
    title_x=0.5
)
fig.show()

In [14]:
from sklearn.svm import SVR
from scipy.stats import uniform as sp_rand
from sklearn.model_selection import RandomizedSearchCV

realized_vol = ret.rolling(5).std() 
realized_vol = pd.DataFrame(realized_vol)
realized_vol.reset_index(drop=True, inplace=True)

In [15]:
returns_svm = ret ** 2
returns_svm = returns_svm.reset_index()
del returns_svm['Date']

In [16]:

X = pd.concat([realized_vol, returns_svm], axis=1, ignore_index=True)
X = X[4:].copy()
X = X.reset_index()
X.drop('index', axis=1, inplace=True)

In [17]:
realized_vol = realized_vol.dropna().reset_index()
realized_vol.drop('index', axis=1, inplace=True)

In [ ]:
svr_lin = SVR(kernel='linear') 
# svr_rbf = SVR(kernel='rbf')

In [ ]:
para_grid = {'gamma': sp_rand(), 'C': sp_rand(), 'epsilon': sp_rand()}
clf = RandomizedSearchCV(svr_lin, para_grid)
clf.fit(X.iloc[:-n].values, realized_vol.iloc[1:-(n-1)].values.reshape(-1,))
predict_svr_lin = clf.predict(X.iloc[-n:])

predict_svr_lin = pd.DataFrame(predict_svr_lin)
predict_svr_lin.index = ret.iloc[-n:].index

rmse_svr = np.sqrt(mse(realized_vol.iloc[-n:] / 100, predict_svr_lin / 100))
print('The RMSE value of SVR with Linear Kernel is {:.6f}'.format(rmse_svr))
mse_svr = mse(realized_vol.iloc[-n:] / 100, predict_svr_lin / 100)
print('The MSE value of SVR with Linear Kernel is {:.7f}'.format(mse_svr))

mae_svr = mae(realized_vol.iloc[-n:] / 100, predict_svr_lin / 100)
print('The MAE value of SVR with Linear Kernel is {:.6f}'.format(mae_svr))

### Pickling the model

In [ ]:
import pickle

In [ ]:
# Saving the file to current directory
with open('model_pickle_svr', 'wb') as f:
    pickle.dump(clf,f)
    

In [ ]:
# Loading the pickle file 
with open('model_pickle_svr','rb') as f:
    mp = pickle.load(f)
    

In [ ]:
# Testing
predict_svr_lin_mp = mp.predict(X.iloc[-n:])
predict_svr_lin_mp = pd.DataFrame(predict_svr_lin_mp)
predict_svr_lin_mp.index = ret.iloc[-n:].index

rmse_svr = np.sqrt(mse(realized_vol.iloc[-n:] / 100, predict_svr_lin_mp / 100))
print('The RMSE value of SVR with Linear Kernel is {:.6f}'.format(rmse_svr))
mse_svr = mse(realized_vol.iloc[-n:] / 100, predict_svr_lin_mp / 100)
print('The MSE value of SVR with Linear Kernel is {:.7f}'.format(mse_svr))

mae_svr = mae(realized_vol.iloc[-n:] / 100, predict_svr_lin_mp / 100)
print('The MAE value of SVR with Linear Kernel is {:.6f}'.format(mae_svr))

In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pickle
import numpy as np

In [10]:
model = keras.Sequential(
    [layers.Dense(256, activation="relu"),
    layers.Dense(128, activation="relu"),
    layers.Dense(1, activation="linear"),])

In [11]:
model.compile(loss='mse', optimizer='rmsprop')

In [12]:
epochs_trial = np.arange(100, 400, 4)
batch_trial = np.arange(100, 400, 4)

In [18]:
DL_pred = []
DL_RMSE = []
DL_MSE = []
DL_MAE = []
for i, j, k in zip(range(4), epochs_trial, batch_trial):
    model.fit(X.iloc[:-n].values, realized_vol.iloc[1:-(n-1)].values.reshape(-1,),
              batch_size=k, epochs=j, verbose=False)
    DL_predict = model.predict(np.asarray(X.iloc[-n:]))
    DL_RMSE.append(np.sqrt(mse(realized_vol.iloc[-n:] / 100, DL_predict.flatten() / 100)))
    DL_MSE.append(mse(realized_vol.iloc[-n:] / 100, DL_predict.flatten() / 100))
    DL_MAE.append(mae(realized_vol.iloc[-n:] / 100, DL_predict.flatten() / 100))
    DL_pred.append(DL_predict)
    print('DL_RMSE_{}:{:.6f}'.format(i+1, DL_RMSE[i]))     
    print('DL_MSE_{}:{:.6f}'.format(i+1, DL_MSE[i]))    
    print('DL_MAE_{}:{:.6f}'.format(i+1, DL_MAE[i]))    

10/10 [==============================] - 0s 4ms/step
DL_RMSE_1:0.005992
DL_MSE_1:0.000036
DL_MAE_1:0.004696
10/10 [==============================] - 0s 2ms/step
DL_RMSE_2:0.003320
DL_MSE_2:0.000011
DL_MAE_2:0.002632
10/10 [==============================] - 0s 2ms/step
DL_RMSE_3:0.004278
DL_MSE_3:0.000018
DL_MAE_3:0.003351
10/10 [==============================] - 0s 3ms/step
DL_RMSE_4:0.004512
DL_MSE_4:0.000020
DL_MAE_4:0.003212


In [19]:
# Dumping model to disk
with open('model_pickle_nn', 'wb') as file:
    pickle.dump(model,file)
    

INFO:tensorflow:Assets written to: ram://383a4d93-0a2f-4f00-86fa-7f4f42734553/assets


In [20]:
# Loading the pickled model
with open('model_pickle_nn','rb') as file:
    mpn = pickle.load(file)

In [21]:
# Testing the pickled model

DL_pred = []
DL_RMSE = []
DL_MSE = []
DL_MAE = []

for i, j, k in zip(range(4), epochs_trial, batch_trial):
    DL_predict = mpn.predict(np.asarray(X.iloc[-n:]))
    DL_RMSE.append(np.sqrt(mse(realized_vol.iloc[-n:] / 100, DL_predict.flatten() / 100)))
    DL_MSE.append(mse(realized_vol.iloc[-n:] / 100, DL_predict.flatten() / 100))
    DL_MAE.append(mae(realized_vol.iloc[-n:] / 100, DL_predict.flatten() / 100))
    DL_pred.append(DL_predict)
    print('DL_RMSE_{}:{:.6f}'.format(i+1, DL_RMSE[i]))     
    print('DL_MSE_{}:{:.6f}'.format(i+1, DL_MSE[i]))    
    print('DL_MAE_{}:{:.6f}'.format(i+1, DL_MAE[i]))    

10/10 [==============================] - 0s 2ms/step
DL_RMSE_1:0.004512
DL_MSE_1:0.000020
DL_MAE_1:0.003212
10/10 [==============================] - 0s 2ms/step
DL_RMSE_2:0.004512
DL_MSE_2:0.000020
DL_MAE_2:0.003212
10/10 [==============================] - 0s 3ms/step
DL_RMSE_3:0.004512
DL_MSE_3:0.000020
DL_MAE_3:0.003212
10/10 [==============================] - 0s 2ms/step
DL_RMSE_4:0.004512
DL_MSE_4:0.000020
DL_MAE_4:0.003212
